In [1]:
import tqdm
import urllib
import os.path
import sqlite3
import pandas as pd
import re
import numpy as np
import requests
import requests_random_user_agent
import threading
from w3lib.url import safe_url_string
from urllib.parse import urlparse, urlunparse
from bs4 import BeautifulSoup as bf


AMAZON_URL = "https://www.amazon.com/s?k={}&i=movies-tv"
AMAZON_URL_HEAD = "https://www.amazon.com"
IMDB_URL =  "https://www.imdb.com/title/tt{}/?ref_=fn_al_tt_1"

MOVIELENS_PATH = './'
OUTPUT_CSV = "./prices.csv"
DATABASE_PATH = "./database.db"
REQUEST_TIMEOUT = 5.0

### Load movies.csv and links.csv

In [2]:
movies_df = pd.read_csv(MOVIELENS_PATH + 'movies.csv', dtype=str)
print(movies_df.head())

links_df = pd.read_csv(MOVIELENS_PATH + 'links.csv',dtype=str)
print(links_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
  movieId   imdbId tmdbId
0       1  0114709    862
1       2  0113497   8844
2       3  0113228  15602
3       4  0114885  31357
4       5  0113041  11862


### Join movies_df and links_df by movieId

In [4]:
merged_df = pd.merge(movies_df, links_df, on='movieId')
print(merged_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres   imdbId tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy  0114709    862  
1                   Adventure|Children|Fantasy  0113497   8844  
2                               Comedy|Romance  0113228  15602  
3                         Comedy|Drama|Romance  0114885  31357  
4                                       Comedy  0113041  11862  


### Create Database and Table

Table structure: movieId (int)，imdbId (varchar30), title (varchar50), pageUrl (varchar300), price (double)

In [2]:
database = sqlite3.connect(DATABASE_PATH, check_same_thread=False)
cursor = database.cursor()
cursor.execute('create table if not exists prices (movieId integer primary key, imdbId varchar(30) NOT NULL, title varchar(50) NOT NULL, pageUrl varchar(300), price double)')
database.commit()

### Query exist tables

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Current table: ", cursor.fetchall())

Current table:  [('prices',)]


### Query table structure

In [6]:
cursor.execute("PRAGMA table_info(prices)")
print("Table prices: ", cursor.fetchall())

Table prices:  [(0, 'movieId', 'integer', 0, None, 1), (1, 'imdbId', 'varchar(30)', 1, None, 0), (2, 'title', 'varchar(50)', 1, None, 0), (3, 'pageUrl', 'varchar(300)', 0, None, 0), (4, 'price', 'double', 0, None, 0)]


### Insert movie data if not exist

In [7]:
row_num = merged_df.shape[0]
pbar = tqdm.trange(row_num, position=0, leave=True)
for i in pbar:
    pbar.set_description("Processing row %d" % i)
    row_item = merged_df.iloc[i]
    cursor.execute("SELECT * FROM prices WHERE movieId = ?;", (int(row_item['movieId']),))
    if len(cursor.fetchall()) == 0:
        cursor.execute("INSERT INTO prices (movieId, imdbId, title, pageUrl, price) VALUES (?, ?, ?, NULL, NULL)",
                       (int(row_item['movieId']), row_item['imdbId'], row_item['title']))
        database.commit()

Processing row 27277: 100%|██████████| 27278/27278 [04:47<00:00, 95.04it/s]


### Display first 5 data from database

In [7]:
cursor.execute("SELECT * FROM prices LIMIT 0, 10;")
print(cursor.fetchall())

[(1, '0114709', 'Toy Story (1995)', 'https://www.amazon.com/Toy-Story-Tim-Allen/dp/B0094KTAEY/ref=sr_1_1?dchild=1&keywords=Toy+Story+1995&qid=1619252106&s=movies-tv&sr=1-1', None), (2, '0113497', 'Jumanji (1995)', 'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619252109&s=movies-tv&sr=1-1', None), (3, '0113228', 'Grumpier Old Men (1995)', 'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619252112&s=movies-tv&sr=1-1', None), (4, '0114885', 'Waiting to Exhale (1995)', 'https://www.amazon.com/Celebrated-Women-Color-Film-Waiting/dp/B003RACH3I/ref=sr_1_1?dchild=1&keywords=Waiting+to+Exhale+1995&qid=1619252113&s=movies-tv&sr=1-1', None), (5, '0113041', 'Father of the Bride Part II (1995)', 'https://www.amazon.com/Father-Bride-Part-Theatrical-Version/dp/B072R146S8/ref=sr_1_1?dchild=1&keywords=Father+of+the+Bride+Part+II+1995&qid=1619252114&s=movies-tv&sr=1-1',

In [41]:
def getPageUrl(title):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    url = safe_url_string(url, encoding="utf-8")
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def getPageUrlFromIMDB(imdbId):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def getMoviePrice(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    lst = [str(x) for x in obj.body.find_all('button') if 'Buy' in str(x)]
    if len(lst) == 0:
        d = obj.body.find('tr', id='priceblock_ourprice_row').find('span', id='priceblock_ourprice').contents
        return float(str(d[0])[1:])
    else:
        return float(re.findall(r'(?<=\$)\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})', lst[0])[0])

def removeBrackets(title):
    return title[:-7] + " " + title[-5:-1]

def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

def iriToUri(iri):
    parts= urlparse(iri)
    return urlunparse(
        part.encode('idna') if parti==1 else urlEncodeNonAscii(part.encode('utf-8'))
        for parti, part in enumerate(parts)
    )

def runGetPage(movie_list):
    pbar = tqdm.tqdm(range(len(movie_list)), position=0, leave=True)
    for i in pbar:
        try:
            pbar.set_description("Processing row %d" % i)
            m = movie_list[i]
            movie_id = m[0]
            movie_title = m[2]
            page_url = m[3]
            if page_url is None:
                page_url = getPageUrl(removeBrackets(movie_title))
                updatePageUrl(page_url, movie_id)
        except KeyboardInterrupt:
            sys.exit()
            pass
        except Exception as e:
            print("Exception at row %d" % i)
            print(e)
            continue

def runGetPrice(movie_list):
    pbar = tqdm.tqdm(range(len(movie_list)), position=0, leave=True)
    for i in pbar:
        try:
            pbar.set_description("Processing row %d" % i)
            m = movie_list[i]
            movie_id = m[0]
            page_url = m[3]
            movie_price = m[4]
            if page_url is not None and movie_price is None:
                price = getMoviePrice(page_url)
                updatePrice(price, movie_id)
        except KeyboardInterrupt:
            sys.exit()
            pass
        except Exception as e:
            print("Exception at row %d" % i)
            print(e)
            continue
            
def getRandomUserAgent():
    s = requests.Session()
    return s.headers['User-Agent']

def list_split(items, n):
    return [items[i:i+n] for i in range(0, len(items), n)]

lock = threading.Lock()

def updatePageUrl(page_url, movie_id):
    try:
        lock.acquire(True)
        cursor.execute("UPDATE prices SET pageUrl = ? WHERE movieId = ?;", (page_url, movie_id))
        database.commit()
    finally:
        lock.release()
        
def updatePrice(price, movie_id):
    try:
        lock.acquire(True)
        cursor.execute("UPDATE prices SET price = ? WHERE movieId = ?;", (price, movie_id))
        database.commit()
    finally:
        lock.release()

In [99]:
cursor.execute("UPDATE prices SET pageUrl = NULL WHERE imdbId = ?;", ("0112682", ))
database.commit()

In [12]:
getPageUrl(removeBrackets("Grumpier Old Men (1995)"))

'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619266492&s=movies-tv&sr=1-1'

In [122]:
imdbId = "0114709"
url = IMDB_URL.format(imdbId)
req = urllib.request.Request(url)
req.add_header('User-Agent', getRandomUserAgent())
html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
obj = bf(html.read(),'html.parser')
print(obj.body.find_all('button',class_="buybox__button"))

[]


### Load data from database to Dataframe

In [ ]:
output_df = pd.DataFrame(columns=['movieId', 'title', 'pageUrl', 'price'])
cursor.execute("SELECT * FROM prices;")
pbar = tqdm.tqdm(cursor.fetchall(), position=0, leave=True)
for row in pbar:
    pbar.set_description("Processing %d" % 1)
    row_dict = {'movieId': row[0], 'title': row[1], 'pageUrl': row[2], 'price': row[3]}
    output_df = output_df.append(row_dict, ignore_index=True)

Processing 1:  77%|███████▋  | 20920/27278 [01:58<00:37, 170.03it/s]

### Write into CSV file

In [121]:
output_df.to_csv(OUTPUT_CSV, index=False)

In [116]:
getPageUrl('Jumanji 1995')

'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619255969&s=movies-tv&sr=1-1'

In [62]:
!pip3 install urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [92]:
from w3lib.url import safe_url_string
url = safe_url_string('http://example.org/Ñöñ-ÅŞÇİİ/', encoding="utf-8")
print(url)

http://example.org/%C3%91%C3%B6%C3%B1-%C3%85%C5%9E%C3%87%C4%B0%C4%B0/


In [106]:
!pip install requests-random-user-agent

     |████████████████████████████████| 47 kB 1.6 MB/s eta 0:00:01


In [15]:
!pip3 install lxml

     |████████████████████████████████| 4.6 MB 609 kB/s eta 0:00:01


In [43]:
import threading
import requests
from lxml import etree
from urllib import request
import os
import re

class PageUrl(threading.Thread):
    def __init__(self, l, *args, **kwargs):
        super(PageUrl, self).__init__(*args,**kwargs)
        self.l = l

    def run(self):
        runGetPage(self.l)
        
class Price(threading.Thread):
    def __init__(self, l, *args, **kwargs):
        super(Price, self).__init__(*args,**kwargs)
        self.l = l

    def run(self):
        runGetPrice(self.l)

In [6]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
PageUrl(movie_list).start()

Processing row 565:   2%|▏         | 474/27278 [00:13<19:48, 22.56it/s]

Exception at row 473
list index out of range


Processing row 1223:   4%|▍         | 1186/27278 [00:44<38:24, 11.32it/s]

In [5]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
WORKERS_NUM = 5
split_list = np.array_split(movie_list, 5)

In [7]:
for i in range(WORKERS_NUM):
    t = PageUrl(split_list[i])
    t.start()

Processing row 27:   0%|          | 22/5456 [00:00<00:49, 110.04it/s]

Exception at row 8
'ascii' codec can't encode character '\xed' in position 23: ordinal not in range(128)
Exception at row 7
'ascii' codec can't encode character '\xc0' in position 9: ordinal not in range(128)
Exception at row 10
'ascii' codec can't encode character '\xe8' in position 33: ordinal not in range(128)
Exception at row 13
'ascii' codec can't encode character '\xe8' in position 40: ordinal not in range(128)
Exception at row 19
'ascii' codec can't encode character '\xda' in position 29: ordinal not in range(128)
Exception at row 20
'ascii' codec can't encode character '\xe1' in position 29: ordinal not in range(128)
Exception at row 21
'ascii' codec can't encode character '\xe4' in position 20: ordinal not in range(128)
Exception at row 24
'ascii' codec can't encode character '\xe7' in position 39: ordinal not in range(128)
Exception at row 23
'ascii' codec can't encode character '\xe8' in position 52: ordinal not in range(128)
Exception at row 28
'ascii' codec can't encode ch

Processing row 98:   1%|▏         | 77/5456 [00:00<00:22, 234.75it/s]]

Exception at row 31
'ascii' codec can't encode character '\xf6' in position 36: ordinal not in range(128)
Exception at row 52
'ascii' codec can't encode character '\xf6' in position 37: ordinal not in range(128)
Exception at row 80
'ascii' codec can't encode character '\xf4' in position 10: ordinal not in range(128)
Exception at row 84
'ascii' codec can't encode character '\xfb' in position 59: ordinal not in range(128)
Exception at row 85
'ascii' codec can't encode character '\xfb' in position 46: ordinal not in range(128)


Processing row 241:   4%|▍         | 213/5456 [00:00<00:13, 398.75it/s]

...

Exception at row 5439
'ascii' codec can't encode character '\xf3' in position 43: ordinal not in range(128)


Processing row 5447: 100%|█████████▉| 5442/5455 [51:01<00:05,  2.31it/s]

Exception at row 5444
'ascii' codec can't encode character '\xe9' in position 26: ordinal not in range(128)


Processing row 5454: 100%|██████████| 5455/5455 [51:05<00:00,  1.78it/s]


Exception at row 5450
'ascii' codec can't encode character '\xe0' in position 17: ordinal not in range(128)


In [6]:
PageUrl(split_list[4]).start()

Processing row 5:   0%|          | 4/5455 [00:08<2:46:55,  1.84s/it]

Exception at row 3
list index out of range
Exception at row 4
'ascii' codec can't encode character '\xeb' in position 33: ordinal not in range(128)


Processing row 11:   0%|          | 11/5455 [00:16<1:38:25,  1.08s/it]

Exception at row 10
list index out of range


Processing row 26:   0%|          | 25/5455 [00:33<1:50:57,  1.23s/it]

...

Processing row 53:   1%|          | 53/5455 [01:12<1:40:35,  1.12s/it]

Exception at row 52
list index out of range


Processing row 80:   1%|▏         | 79/5455 [01:55<2:02:42,  1.37s/it]

Exception at row 79
'ascii' codec can't encode character '\xe1' in position 16: ordinal not in range(128)


Processing row 86:   2%|▏         | 86/5455 [01:59<1:36:29,  1.08s/it]

In [3]:
cursor.execute('SELECT * FROM prices WHERE pageUrl IS NOT NULL')
len(cursor.fetchall())

16054

In [6]:
cursor.execute('SELECT * FROM prices')
len(cursor.fetchall())

27278

In [58]:
cursor.execute('SELECT * FROM prices WHERE pageUrl IS NOT NULL')
len(cursor.fetchall())

Processing row 2912:  53%|█████▎    | 2912/5455 [1:07:56<1:03:31,  1.50s/it]

Exception at row 2911
'NoneType' object has no attribute 'find'


26475

Processing row 3681:  67%|██████▋   | 3681/5456 [1:07:57<56:59,  1.93s/it]

Exception at row 3680
'NoneType' object has no attribute 'find'


...

Processing row 3241:  59%|█████▉    | 3241/5455 [1:08:13<57:29,  1.56s/it]  

Exception at row 3240
'NoneType' object has no attribute 'find'


Processing row 3244:  59%|█████▉    | 3242/5455 [1:08:13<43:06,  1.17s/it]

Exception at row 3241
'NoneType' object has no attribute 'find'


Processing row 3245:  59%|█████▉    | 3245/5455 [1:08:13<31:05,  1.18it/s]

In [22]:
getMoviePrice("https://www.amazon.com/Superbad-Unrated-Jonah-Hill/dp/B00EFCHT08/ref=sr_1_1?dchild=1&keywords=Superbad+2007&qid=1619260061&s=movies-tv&sr=1-1")

Processing row 5387:  99%|█████████▉| 5387/5455 [2:00:20<00:20,  3.28it/s]

Exception at row 5386
list index out of range


Processing row 5388:  99%|█████████▉| 5388/5455 [2:00:20<00:19,  3.40it/s]

Exception at row 5387
list index out of range


IndexError: list index out of range

Processing row 5392:  99%|█████████▉| 5390/5455 [2:00:22<00:36,  1.80it/s]

Exception at row 5389
list index out of range
Exception at row 5390
'ascii' codec can't encode character '\xfc' in position 10: ordinal not in range(128)
Exception at row 5391
'ascii' codec can't encode character '\xfc' in position 20: ordinal not in range(128)


In [46]:
lst = [str(x) for x in obj.body.find_all('button') if 'Buy' in ]

In [58]:
re.findall(r'(?<=\$)\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})', "$6.99")

['6.99']

In [44]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
WORKERS_NUM = 5
split_list = np.array_split(movie_list, 5)

In [45]:
for i in range(WORKERS_NUM):
    t = Price(split_list[i])
    t.start()

Processing row 10:   0%|          | 4/5456 [00:02<1:03:38,  1.43it/s]

Exception at row 3
'NoneType' object has no attribute 'find'


Processing row 1:   0%|          | 1/5455 [00:03<4:40:28,  3.09s/it]

Exception at row 0
'NoneType' object has no attribute 'find'


Processing row 4:   0%|          | 4/5456 [00:03<1:16:06,  1.19it/s]

...

Exception at row 28
'NoneType' object has no attribute 'find'
Exception at row 4
'NoneType' object has no attribute 'find'


Processing row 15:   0%|          | 14/5456 [00:20<3:16:36,  2.17s/it]

Exception at row 13
'NoneType' object has no attribute 'find'


Processing row 36:   1%|          | 30/5456 [00:20<1:46:51,  1.18s/it]

Exception at row 29
'NoneType' object has no attribute 'find'


'<button class="_1ovr-S _15Ikr8 _20uSE4 _2X_Irl tvod-button av-button" type="submit"><span class="_36qUej">Buy <span class="dv-conditional-linebreak"><br/></span><strong>HD</strong> $6.99</span></button>'

In [66]:
cursor.execute('SELECT * FROM prices WHERE price IS NOT NULL')
len(cursor.fetchall())

11434

In [42]:
url = "https://www.amazon.com/Jacky-Kingdom-royaume-filles-Blu-Ray/dp/B00IYYPI38/ref=tmm_blu_swatch_0?_encoding=UTF8&qid=1619285344&sr=1-1"
getMoviePrice(url)

23.98

In [65]:
removeBrackets("Citizen Gangster (2011)")

'Citizen Gangster 2011'

Processing row 4775:  88%|████████▊ | 4775/5455 [1:47:44<26:06,  2.30s/it]

Exception at row 4774
'NoneType' object has no attribute 'find'


Processing row 4744:  87%|████████▋ | 4743/5456 [1:47:44<32:30,  2.74s/it]

Exception at row 4742
'NoneType' object has no attribute 'find'


...

Exception at row 5452
'NoneType' object has no attribute 'find'


Processing row 5454: 100%|█████████▉| 5454/5455 [2:03:51<00:01,  1.57s/it]

Exception at row 5453
'NoneType' object has no attribute 'find'


Processing row 5454: 100%|██████████| 5455/5455 [2:03:52<00:00,  1.36s/it]

Exception at row 5454
'NoneType' object has no attribute 'find'
